In [1]:
import pandas as pd
df = pd.read_csv('/home/myproject/인천서구.csv')
df.head()

,Title,Date,Content,Link
0,매매도 전세도 수도권만 올라…지방은 하락 계속,2023-07-13 14:02:00,전국 아파트값 3주 연속 보합…서울은 8주 연속 상승\n지방은 낙폭 못 줄이고 하락...,https://n.news.naver.com/mnews/article/079/000...
1,"‘호반써밋파크에디션’, 18일 1순위 청약 접수",2023-07-13 14:01:00,인천 연희공원 내 지하2층 ~지상34층\n전용84~99㎡ 총 1370가구 규모\n호...,https://n.news.naver.com/mnews/article/277/000...
2,전국 아파트값 3주 연속 '보합'…수도권↑·지방↓,2023-07-13 14:00:00,"서울은 강남, 강북 모두 올라\n전세도 수도권·서울만 상승…지방은 하락\n서울 송파...",https://n.news.naver.com/mnews/article/421/000...
3,'호우주의보' 인천 도로 침수 잇따라…하천 3곳 등 통제(종합),2023-07-13 13:55:00,"시, 지대본 1단계 운영, 인력 투입 비상근무 가동\n전국 대부분 지역에 장맛비가 ...",https://n.news.naver.com/mnews/article/421/000...
4,송도 72.5㎜…'호우주의보' 인천 도로 침수 잇따라,2023-07-13 13:46:00,13일 낮 12시10분께 중구 덕교동 덕교삼거리에 도로가 침수됐다는 신고가 접수돼 ...,https://n.news.naver.com/mnews/article/421/000...


In [2]:
import MeCab
from gensim.models import FastText
from gensim.models.keyedvectors import KeyedVectors
from collections import defaultdict
from operator import itemgetter
import pandas as pd

# MeCab 형태소 분석기 초기화
m = MeCab.Tagger()

# 특정 컬럼의 텍스트 데이터 가져오기 (실제 컬럼 이름으로 대체)
text_data = df['Content'].tolist()

# 명사 추출을 위한 함수
def extract_nouns(text):
    parsed_result = m.parse(text)
    lines = parsed_result.split('\n')
    nouns = []
    for line in lines:
        if '\t' in line:
            word, info = line.split('\t')
            pos_info = info.split(',')[0]
            if pos_info == 'NNG' and len(word) > 1:  # 한 글자 명사 제거
                nouns.append(word)
    return nouns

# 각 문장에 대해 명사만 추출하여 리스트로 저장
nouns_list = [extract_nouns(sentence) for sentence in text_data]

# FastText 모델 학습
model = FastText(sentences=nouns_list, size=100, window=5, min_count=1, sg=1)

# 학습된 모델 저장
model.save('fasttext_model.bin')

In [3]:
from gensim.models import FastText

# FastText 모델 로드 (unsupervised)
model_path = 'fasttext_model.bin'  # 사전학습된 FastText 모델 파일 경로
fasttext_model = FastText.load(model_path)

related_words = fasttext_model.wv.most_similar('인천 서구', topn=3)

related_words

[('서구', 0.9953876733779907),
 ('중구', 0.7615519165992737),
 ('영종', 0.7450915575027466)]